# Atelier 1 : Denoising parfaitement supervisé

In [1]:
# Pour commencer : aller dans éxecution/modifier le type d'exécution et vérifier
# que CPU est bien coché (on n'a pas besoin de plus pour l'instant)

# Imports des bibliothèques utiles
# pour l'IA
import torch
# pour les maths
import numpy as np
# pour afficher des images et des courbes
import matplotlib.pyplot as plt

In [ ]:
! git clone https://github.com/nanopiero/PREAC.git

In [ ]:
! ls PREAC

## A. Découverte du problème

In [ ]:
from PREAC.utile_denoising import gen, voir_batch2D

input, target = gen(6)

# Entrées (images bruitées)
fig0 = plt.figure(0, figsize=(36, 6))
voir_batch2D(input, 6, fig0, k=0, min_scale=0, max_scale=1)

# Cibles (images non bruitées)
fig1 = plt.figure(1, figsize=(36, 6))
voir_batch2D(target, 6, fig1, k=0, min_scale=0, max_scale=1)

## Question intéressante ##
Sous quelle forme les images sont-elles représentées ?


## B.Prise en main de l'outil

Instantiation d'un réseau U-Net :

In [6]:
from PREAC.utile_denoising import UNet

ch_in = 1
ch_out = 1
size = 16

# Instanciation :
fcn = UNet(ch_in, ch_out, size)

**Questions intéressantes :**  \\
Où le réseau est-il codé ? \\
Combien y a-t-il de poids (taille de $\theta$) dans le réseau ? \\
Que représentent *ch_in*, *ch_out*, *size* ? \\
Comment l'appliquer à une image ? \\
Quelle sont alors les dimensions de l'output ? \\
Comment visualiser l'output ?

## C.Entraînement sur CPU

In [4]:
# Pour comparer la sortie et la cible:
def Loss(output, target):
    return torch.mean((output - target)**2)

In [7]:
# Pour définir le type de descente de gradient:
import torch.optim as optim
optimizer = optim.Adam(fcn.parameters(), 10**(-4))

In [ ]:
nepochs = 1 # Unité de l'apprentissage (normalement: parcours complet du jeu de données)
nbatches = 50  # mini-batches par époque
batchsize = 32  # taille d'un mini-batch

train_losses = []  # Liste qui contiendra les écarts moyens sur l'époque

for epoch in range(nepochs):
    print(f"Epoch {epoch + 1}/{nepochs}")

    epoch_losses = []  # Liste auxiliaire, pour stocker les écarts

    for i in range(nbatches):

        inputs, targets = gen(batchsize)

        optimizer.zero_grad()
        outputs = fcn(inputs)
        loss = Loss(outputs, targets)
        loss.backward()
        optimizer.step()

        epoch_losses.append(loss.detach().cpu().item())

    epoch_loss = np.mean(epoch_losses)
    train_losses.append(epoch_loss)

    print(f'Epoch loss: {epoch_loss:.4f}')

## Questions intéressantes ##
A quoi sert chaque ligne ? \\
Combien de temps prend une époque ? \\
Comment accélerer le calcul ?

# D. Entraînement sur carte GPU

In [10]:
# Passer un objet sur l'accélérateur :
device = torch.device('cuda:0')
input = input.to(device)
fcn = fcn.to(device)

# Reprendre l'entraînement

## Questions intéressantes ##
Quel est le gain de temps sur GPU ? \\
Comment l'apprentissage a-t-il progressé ? \\
Pourquoi n'utilisons-nous qu'un seul jeu ici (et pas trois) ? \\
Quels sont les outputs après 10 époques, 50 époques ? \\
Une régression linéaire pixel à pixel ne ferait-elle pas l'affaire ?
